# Web Scraping IWF Website For Phuket 2024 Event

In [ ]:
# Import libraries required for task
import pandas as pd
from functools import reduce

from bs4 import BeautifulSoup, NavigableString, Tag

import requests
import re


from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 

## Resources Used
- 2024.09.12/ChatGPT accessed
- 2024.09.11/[Repository](https://github.com/jwc20/iwf_api/blob/main/iwf/result.py) accessed

## Table of Contents

- [IWF Web Scraping](#IWF-Web-Scraping)
- [Clean & Feature Engineer Scraped Data](#Clean-&-Feature-Engineer-Scraped-Data)
- [Merge With Provided_Data](#Merge-With-Provided-Data)
- [Verify Merge and Write To CSV](#Verify-Merge-and_Write-To-CSV)

## IWF Web Scraping

In [ ]:
# Call the URL and get a response
url = 'https://iwf.sport/results/results-by-events/?event_id=599'
response = requests.get(url)

In [ ]:
# Success!
response

In [ ]:
#Turn the response into html text, then parse with BeautifulSoup
html = response.text
soup = BeautifulSoup(html, 'lxml')

#### I only need data from the Men and Women's Snatch & Clean and Jerk.  The HTML is structured to have a 'result__container' and each are labeled with one of the four different categories.

In [ ]:
# Find all of the result__container div classes and verify that each one has their own category.
result_containers = soup.find_all('div', {'class':'result__container'})

for id in result_containers:
    print(id.get('id'))

#### The following section will pull requested project data from the 2024 IWF World Cup Paris Qualifying Event in Phuket and consolidate data into one dataframe.
- Looks for result_containers that have either the 'men_snatchjerk' or 'women_snatchjerk' ID.
- Pulls all of the div tags with the 'cards' class and put them into a 'cards_group' variable.
- Looks for all div classes with the 'card' class within the 'cards_group' variable and puts them into a 'card_box' variable.
- Runs three 'for' loops to collect snatch, clean and jerk, and total lift data.
- Collects all of the data into three separate dataframes for snatch, clean and jerk, and total lift data, then merges on competitor name into one dataframe.

In [ ]:
# Create empty lists for each requested lift category so they can later be transformed into dataframes
snatch_list = []
cj_list = []
total_list = []

# For loop check if the div ID of the result__container and if satisfies the criteria, it finds all of the divs with the 'cards' class and 
# put it into a cards_group varaible.

for category in result_containers:
    if category.get('id') == 'men_snatchjerk' or category.get('id') == 'women_snatchjerk':
        cards_group = category.find_all('div', {'class':'cards'})
        
# SNATCH DATA SCRAPE
# Following pair of for loops iteratres through the cards_group associated with snatch data (every third group starting with first group) and puts
# it into a 'card_box' variable.  For every card in the box, except the first one because it just contains column labels, second loop will check for
# specific 'p' tags that hold requested data to scrape.  Pulling name for each category of lifts so that they can later be merged. Creates key
# value pairs for data points and appends them to list.
        
        for cards in cards_group[::3]:
            card_box = cards.find_all('div', {'class':'card'})

            for card in card_box[1:]:
                snatch_data = {}
                snatch_data['name'] = card.find_all('p')[1].text.strip()
                snatch_data['country'] = card.find_all('p')[2].text.strip()
                snatch_data['snatch1'] = card.find_all('p')[6].strong.contents[0]
                snatch_data['snatch2'] = card.find_all('p')[7].strong.contents[0]
                snatch_data['snatch3'] = card.find_all('p')[8].strong.contents[0]
                snatch_list.append(snatch_data)

# CLEAN AND JERK DATA SCRAPE
# Does the same as the snatch data scrape except that it looks through each clean and jerk weight category (every third group starting with
# the second group.
        
        for cards in cards_group[1::3]:
            card_box = cards.find_all('div', {'class':'card'})

            for card in card_box[1:]:
                cj_data = {}
                cj_data['name'] = card.find_all('p')[1].text.strip()
                cj_data['cj1'] = card.find_all('p')[6].strong.contents[0]
                cj_data['cj2'] = card.find_all('p')[7].strong.contents[0]
                cj_data['cj3'] = card.find_all('p')[8].strong.contents[0]
                cj_list.append(cj_data)

# SUM LIFT DATA SCRAPE
# Does the same as the snatch and clean and jerk data scrapes.  Sum lift is the combined sum of an athlete's best snatch and clean and jerk
# successful lifts.  

        for cards in cards_group[2::3]:
            card_box = cards.find_all('div', {'class':'card'})

            for card in card_box[1:]:
                sum_data = {}
                sum_data['name'] = card.find_all('p')[1].text.strip()
                sum_data['total'] = card.find_all('p')[8].strong.contents[1]
                total_list.append(sum_data)

# Transforms list data for each lift category into dataframes

snatch_df = pd.DataFrame(snatch_list)
cj_df = pd.DataFrame(cj_list)
total_df = pd.DataFrame(total_list)

# Combines all three dataframes into one, merging on the 'name' column using the reduce function

dfs = [snatch_df, cj_df, total_df]
iwf_df = reduce(lambda left, right: pd.merge(left, right, on = 'name'), dfs)

## Clean & Feature Engineer Scraped Data

#### Requested data has been scraped, but needs a bit more clean-up and feature engineering: Add columns to indicate if a lift was success or not as well as edit lift columns so they can be integers for potential calculations in the future.

In [ ]:
iwf_df.head(10)

#### The values are either a NavigableString or a Tag since the values were parsed through BeautifulSoup.  First a function will be created to check if an athlete was successful with a lift and create Boolean colums to reflect results. Another function will transform the weight data into integers and turn any '---' values into 0.

In [ ]:
def lift_success_check(weight):
    '''
    Accepts a weight value from one of the snatch or clean and jerk categories. Checks if a lift weight is a NavigableString and does not
    equal '---'.  If statements are satisfied, the function returns True.  Otherwise it returns False.  This is meant to check if the
    athlete successfully lifted the weight.  If it's in brackets (a Tag) or is '---', that means the athelete was either not able
    to lift weight or didn't participate.
    '''
    return type(weight) == NavigableString and weight != '---'

def int_transformer(weight):
    '''
    Accepts a weight value from one of the snatch or clean and jerk categories.  Checks if the value does not equal '---'.  If it does not, the
    function will change the value into an integer.  Otherwise it replaces '---' with 0.  This is to antipicate any need for calculations with
    the lift weight values.
    '''
    
    if weight != '---':
        return int(weight.text)
    else:
        return 0
        

In [ ]:
# Creating columns lists so that functions can be applied to them.  Separate list for the int_transformer column
# since there is no success or fail metric or total lift, but it needs to be transformed into an integer.
lift_columns = ['snatch1', 'snatch2', 'snatch3', 'cj1', 'cj2', 'cj3']
lift_and_total_columns = ['snatch1', 'snatch2', 'snatch3', 'cj1', 'cj2', 'cj3', 'total']

In [ ]:
# Above functions applied to dataframe
for column in lift_columns:
    iwf_df[f'{column}_success'] = iwf_df[column].apply(lift_success_check)

for column in lift_and_total_columns:
    iwf_df[column] = iwf_df[column].apply(int_transformer)

In [ ]:
iwf_df.head()


In [ ]:
iwf_df.to_csv('../data/phuket_2024.csv', index=False)

In [ ]:
iwf_df.info()

## Merge With Provided Data

In [ ]:
weightlift_df = pd.read_excel('../data/weightlifting_data.xlsx')

In [ ]:
# Noticed the 'PersonID' was sometimes reading as dates in Excel, so changed the column from 'general' to 'number' with no decimals.
weightlift_df.head()

#### Transforming names to snake case

In [ ]:
# Not strictly necessary, but I prefer snake case.
def snake_case(title):
    pattern = re.compile(r'(?<!^)(?=[A-Z])')
    return pattern.sub('_', title).lower()

In [ ]:
weightlift_df.columns = [snake_case(column) for column in weightlift_df.columns]

In [ ]:
# Not perfect - EventID turned into event_i_d, but will keep
weightlift_df.head()

#### I found discrepancies in competitor name structures between the website data and the provided Excel sheet.  

#### Initially, Fuzzy Wuzzy was used for name matching; however, some matches were found to be incorrect. To improve accuracy, competitor country data was scraped from the IWF website and used as a confirmation alongside fuzzy matching. The function below calculates a 'best_score' for each name comparison and checks if the country codes match.  

#### If the country codes match, a bonus is added to name_score. The function selects the highest-scored name from the IWF DataFame to match with the weightlifting dataFrame. Finally, the datasets are merged based on the best matching name, and the resulting dataset is cleaned.

In [ ]:
iwf_df.head()

In [ ]:
weightlift_df.head()

In [ ]:
def fuzzy_name_merge(df_1, df_2, name_col1, name_col2, country_col1, country_col2):
    '''
    df1 = First dataframe to merge
    df2  = Second dataframe to merge
    name_col1 = Column of names from df1
    name_col2 - Column of names for df2
    country_col1 = Column of countries from df1
    country_col2 = Column of countries from df2

    Function makes a copy of df1 and execute 'find_best_match' function.
    It will then unzip tuple return data from 'find_best_match' function and assign them to 'best_match' and 'match_score' columns.
    Function will merge 'merged_df'(copy of df1) with df2 on the best_match column.
    Several columns are dropped and renamed as needed. Columns are reorganized before being returned.
    '''
    # Create a copy of df_1(weightlift_df)
    merged_df = df_1.copy()

    # Function finds the best match based on fuzzy matching combined with competitor country matches
    def find_best_match(row, candidates):
        '''
        row = Row from 'merged_df'(copy of df1 from fuzzy_name_merge function)
        candidates = df2 from fuzzy_name_merge function

        Function will first compare the country from df1 with country from df2 row by row.  It will then use fuzzy's token set ratio
        to compare the current row's name in df1 with names in df2 and assign a score.  If the countries also match, the function
        will assign a bonus.  This will happen for each row in df1 and the top score and name associated with the score is tracked.
        The name with the highest score along with its score is return as a tuple.
        '''
        name = row[name_col1]
        country = row[country_col1]
        best_match = None
        best_score = 0
        for index, candidate in candidates.iterrows():
            # Check if countries match
            country_match = (country == candidate[country_col2])
            
            # Compare and score names from weightlifting and iwf datasets based on similiarity
            name_score = fuzz.token_set_ratio(name.lower(), candidate[name_col2].lower())
            
            # Boost score if countries match
            adjusted_score = name_score + 20 if country_match else name_score

            # Check if current calculated score is the best one so far and update 'best_score' and associated name with that score
            if adjusted_score > best_score:
                best_score = adjusted_score
                best_match = candidate[name_col2]
        
        return best_match, best_score
        

    # Run fuzzy and country matching function
    m = merged_df.apply(lambda row: find_best_match(row, df_2), axis=1)

    # Unzip best_match and match_score from m and add them as columns to merged_df
    merged_df['best_match'], merged_df['match_score'] = zip(*m)

    # Merge df_1 with df_2 based on the best_match
    merged_df = pd.merge(merged_df, df_2, left_on='best_match', right_on=name_col2, how='left')

    # Drop columns that are no longer needed and rename country column
    merged_df = merged_df.drop(['competitor', 'country_y', 'best_match', 'match_score'], axis=1)
    merged_df = merged_df.rename(columns = {'country_x':'country'})

    # Reorder the columns to  visually make more sense
    merged_df = merged_df.reindex(columns = ['sport',
                                             'season',
                                             'competition_name',
                                             'event_i_d',
                                             'event_name_short',
                                             'event_gender',
                                             'competition_date',
                                             'name',
                                             'person_i_d',
                                             'person_age_days',
                                             'country',
                                             'rank',
                                             'snatch1',
                                             'snatch1_success',
                                             'snatch2',
                                             'snatch2_success',
                                             'snatch3',
                                             'snatch3_success',
                                             'cj1',
                                             'cj1_success',
                                             'cj2',
                                             'cj2_success',
                                             'cj3',
                                             'cj3_success',
                                             'total'])
    return merged_df


In [ ]:
# Run the above code to merge dataframes based on fuzzy matches names along with countries
final_df = fuzzy_name_merge(weightlift_df, iwf_df, 'competitor', 'name', 'country', 'country')

## Verify Merge and Write To CSV

In [ ]:
final_df.head()

In [ ]:
# Write merged data back into Excel file
final_df.to_csv('../data/final_data.csv', index = False)